In [15]:
import os
import random
import re
import time
from datetime import datetime
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from IPython.display import clear_output, display
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
# from seleniumwire import webdriver
import pickle
import time
from ast import literal_eval
from selenium_stealth import stealth
import undetected_chromedriver as uc
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import urllib.parse
from concurrent.futures import ThreadPoolExecutor
import logging
from queue import Queue

In [3]:
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    handlers=[logging.FileHandler('app.log')], encoding="utf-16")
logger = logging.getLogger(__name__)

In [4]:
approximate_reviews_count = 150 
product_counts = 15
sleep_time = 0.5
reviews_per_load = 30
waiting_time_product = 0.5
driver_wait_time = 1

scrolls_count = approximate_reviews_count / 1.2

In [5]:
def get_product_links_category_name(
    driver, category_link, waiting_time_product, product_counts
):
    try:
        driver.get(category_link)

        category_name_element = driver.find_elements(By.CSS_SELECTOR, ".breadcrumbs__item")
        category_name = " / ".join([cn.text for cn in category_name_element[1:]])
        category_products = driver.find_elements(
            By.CSS_SELECTOR, ".product-card__link.j-card-link.j-open-full-product-card"
        )
        products_urls = [elem.get_attribute("href") for elem in category_products]
        time.sleep(waiting_time_product)
        product_ids = [re.findall(r"(\bcatalog\b)\/(\d+)", u)[0][1] for u in products_urls][
            :product_counts
        ]
        return (
            [f"https://www.wildberries.ru/catalog/{id}/feedbacks" for id in product_ids],
            category_name,
        )
    except Exception as e:
        logger.error(e)

In [6]:
def scroll_parse_reviews(
    driver,
    url,
    scrolls_count=100,
    waiting_time_product=10,
    category="unknown",
):
    res = []

    try:
        driver.get(url)

        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, ".product-line__name.hide-mobile")
            )
        )

        elements_categories = driver.find_elements(By.CLASS_NAME, "sorting__item")
        driver.implicitly_wait(1)
        if elements_categories:
            elements_categories[-1].click()
        # Подгружаем отзывы
        for i in range(int(scrolls_count)):
            driver.execute_script(f"window.scrollBy(0, {random.randrange(200,400)});")
            time.sleep(random.randrange(1, 4) / 100)
        driver.implicitly_wait(driver_wait_time)
        # Парсим подгруженную страницу отзывов
        htmls = []
        htmls.append(driver.page_source)
        
        # Парсинг "этот вид товара"
        # try:
        #     driver.execute_script("window.scrollTo(0, 0);")
        #     driver.find_elements(By.CLASS_NAME, "product-feedbacks__title")[-1].click()
        #     WebDriverWait(driver, 60).until(
        #         EC.presence_of_element_located(
        #             (By.CSS_SELECTOR, ".product-line__name.hide-mobile")
        #         )
        #     )
        #     for i in range(int(scrolls_count)):
        #         driver.execute_script(f"window.scrollBy(0, {random.randrange(200,400)});")
        #         time.sleep(random.randrange(1, 4) / 100)
        #     htmls.append(driver.page_source)
        # except:
        #     pass
        for html in htmls:
            soup = BeautifulSoup(html)
            reviews = soup.findAll("li", class_="comments__item feedback j-feedback-slide")
            product_name = driver.find_element(
                By.CSS_SELECTOR, ".product-line__name.hide-mobile"
            ).text

            for review in reviews:
                text = review.find("p", class_="feedback__text")
                if text:
                    text = "".join(text.find_all(text=True, recursive=False))
                title_element = review.find(
                    "span",
                    class_=lambda value: value
                    and value.startswith("feedback__rating stars-line"),
                )
                if text:
                    res.append(
                        {
                            "review_full_text": text,
                            "review_rating": title_element["class"][-1][-1],
                            "product": product_name,
                            "category": category,
                            "url": driver.current_url,
                        }
                    )
            soup.decompose()
        time.sleep(waiting_time_product)
    except Exception as e:
        logger.error(e)
    finally:
        return res

In [7]:
from fake_useragent import UserAgent

chrome_options = Options()

# chrome_options.add_argument("user-agent=Chrome/91.0.4472.124")
chrome_options.add_argument("--disable-blink-features")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("disable-infobars")
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("detach", True)

chrome_options.add_argument('window-size=1920x1080')
# chrome_options.add_argument("disable-gpu")

chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--disable-software-rasterizer")
chrome_options.add_argument("--disable-background-timer-throttling")
chrome_options.add_argument("--disable-backgrounding-occluded-windows")

chrome_options.page_load_strategy = 'eager'


chrome_options.add_argument('headless')

In [8]:
url = "https://static-basket-01.wbbasket.ru/vol0/data/main-menu-ru-ru-v2.json"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    logger.error(f"Ошибка: {response.status_code}")

In [9]:
# Функция для рекурсивного извлечения всех записей
def extract_records(data, parent_id=None, category=""):
    records = []
    for item in data:
        record = {key: value for key, value in item.items() if key != 'childs'}
        record['parent_id'] = parent_id
        record['category'] = "/".join([category, record['name']])
        records.append(record)
        if 'childs' in item:
            try:
                records.extend(extract_records(item['childs'], parent_id=item['id'], category=record['category']))
            except:
                k = 0
    return records

# Извлечение всех записей
all_records = extract_records(data)

In [10]:
def add_substring_if_missing(s):
    if "http" not in s:
        return "https://www.wildberries.ru" + s
    return s

df_categories = pd.DataFrame.from_dict(all_records)
# Применение функции к столбцу 'text'
df_categories['url'] = df_categories['url'].apply(add_substring_if_missing)
df_categories

,id,name,url,parent_id,category,shard,query,isDenyLink,parent,seo
0,1111,ЖАР-СКИДКИ,https://www.wildberries.ru/promotions/zhar-dni...,NaN,/ЖАР-СКИДКИ,NaN,NaN,NaN,NaN,NaN
1,1234,Сертификаты Wildberries,https://www.wildberries.ru/gift/certificates,NaN,/Сертификаты Wildberries,NaN,NaN,NaN,NaN,NaN
2,306,Женщинам,https://www.wildberries.ru/catalog/zhenshchinam,NaN,/Женщинам,blackhole,cat=306,True,NaN,NaN
3,8126,Блузки и рубашки,https://www.wildberries.ru/catalog/zhenshchina...,306.0,/Женщинам/Блузки и рубашки,bl_shirts,cat=8126,NaN,306.0,Женские блузки и рубашки
4,8127,Брюки,https://www.wildberries.ru/catalog/zhenshchina...,306.0,/Женщинам/Брюки,pants,cat=8127,NaN,306.0,Женские брюки
...,...,...,...,...,...,...,...,...,...,...
2202,130630,Автобусы,https://vmeste.wildberries.ru/bus?utm_source=w...,61037.0,/Путешествия/Автобусы,NaN,NaN,NaN,61037.0,NaN
2203,130631,Маршруты,https://vmeste.wildberries.ru/ammp?utm_source=...,61037.0,/Путешествия/Маршруты,NaN,NaN,NaN,61037.0,NaN
2204,130851,Отели,https://vmeste.wildberries.ru/hotel?utm_source...,61037.0,/Путешествия/Отели,NaN,NaN,NaN,61037.0,NaN
2205,130628,Регионы,https://vmeste.wildberries.ru/adventure,61037.0,/Путешествия/Регионы,NaN,NaN,NaN,61037.0,NaN


In [11]:
def encode_forbidden_chars(folder_name):
    forbidden_chars = '<>:"/\\|?*'
    encoded_name = ""
    for char in folder_name:
        if char in forbidden_chars:
            encoded_name += urllib.parse.quote(char)
        else:
            encoded_name += char
    return encoded_name

In [12]:
def parse_site(df_iterator, driver):
    i, df_row = df_iterator
    all_reviews = []
    links, category_name = get_product_links_category_name(
        driver, df_row["url"], waiting_time_product, product_counts
    )
    for url in links:
        reviews = scroll_parse_reviews(
            driver,
            url,
            scrolls_count,
            waiting_time_product,
            category=category_name,
        )
        if reviews:
            all_reviews.extend(reviews)

    folder_name = "./wbdirs" + df_row["category"]
    encoded_folder_name = encode_forbidden_chars(folder_name)
    os.makedirs(encoded_folder_name, exist_ok=True)
    logger.info(f"Category № {i} is parsed: {df_row['category']}")
    if all_reviews:
        df = pd.DataFrame.from_dict(all_reviews)
        df["category"] = df_row["category"]
        dt = datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S")
        file_path = os.path.join(
            os.path.abspath(encoded_folder_name), f"unprepared_{dt}.csv"
        )
        df.to_csv(file_path, encoding="UTF-8", index=False)
    else:
        file_path = os.path.join(os.path.abspath(encoded_folder_name), ".empty")
        with open(file_path, "w") as file:
            file.write("empty")
    all_reviews = []

In [14]:
# Number of drivers
drivers_num = 4
# Define your make_driver function
def make_driver():
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(driver_wait_time)
    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True)
    return driver
# Create drivers
drivers = [make_driver() for _ in range(drivers_num)]

KeyboardInterrupt: 

In [ ]:

# Create a queue and add tasks
task_queue = Queue()
for row in df_categories[20:1200].iterrows():
    task_queue.put(row)

In [ ]:
# Worker function to process tasks
def worker(driver):
    while not task_queue.empty():
        row = task_queue.get()
        try:
            parse_site(row, driver)
        finally:
            task_queue.task_done()

In [ ]:
to_wait = (
    ((waiting_time_product + scrolls_count * 0.025 * 1.1))
    * product_counts
    * task_queue.qsize()
    / len(drivers)
)*2.1
logger.info(f"Approximate waiting time: {round(to_wait)}s")

In [ ]:
# Create and start threads
with ThreadPoolExecutor(max_workers=drivers_num) as executor:
    futures = [executor.submit(worker, driver) for driver in drivers]

# Wait for all tasks to be completed
task_queue.join()

# Quit all drivers
for driver in drivers:
    driver.quit()

C:\Users\Kirill\AppData\Local\Temp\ipykernel_3680\3499826093.py:57: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text = "".join(text.find_all(text=True, recursive=False))


In [16]:
# Function to get all CSV files in a directory and its subdirectories
def get_all_csv_files(directory):
    csv_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files

# Function to read all CSV files into a single DataFrame
def read_csv_files_to_dataframe(csv_files):
    dataframes = []
    for file in csv_files:
        df = pd.read_csv(file)
        dataframes.append(df)
    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df

# Directory containing the CSV files
directory = 'wbdirs'

# Get all CSV files
csv_files = get_all_csv_files(directory)

# Read CSV files into a single DataFrame
combined_df = read_csv_files_to_dataframe(csv_files)

# Display the combined DataFrame
combined_df


,review_full_text,review_rating,product,category,url
0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
4,"Всё в комплекте, есть инструкция на русском яз...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
...,...,...,...,...,...
2968356,Купила себе на новый год. Люблю перебирать чет...,5,Фабрика-Ф / Четки мусульманские серебро серебр...,/Ювелирные изделия/Четки,https://www.wildberries.ru/catalog/16383687/fe...
2968357,"Очень классные чётки! Приятно пользоваться, де...",5,Фабрика-Ф / Четки мусульманские серебро серебр...,/Ювелирные изделия/Четки,https://www.wildberries.ru/catalog/16383687/fe...
2968358,Мило,5,Фабрика-Ф / Четки мусульманские серебро серебр...,/Ювелирные изделия/Четки,https://www.wildberries.ru/catalog/16383687/fe...
2968359,Даже звезду не хочу ставить - пластмасса краше...,1,Фабрика-Ф / Четки мусульманские серебро серебр...,/Ювелирные изделия/Четки,https://www.wildberries.ru/catalog/16383687/fe...


In [24]:
# combined_df.drop_duplicates().to_csv("./data/wildberries_reviews.csv.gz", compression='gzip')
# pd.read_csv('./data/wildberries_reviews.csv.gz', compression='gzip')
# combined_df.to_csv("./data/wildberries_reviews.csv")
# pd.read_csv("./data/wildberries_reviews.csv")